In [1]:
import pandas as pd
import numpy as np
import time
import os
import zipfile
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [3]:
query_reserva = """ SELECT * FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_militares_reserva_reforma_cadastro`
            """ 

In [4]:
df_reserva = pandas_gbq.read_gbq(query_reserva, project_id='repositoriodedadosgpsp')
df_reserva

Downloading: 100%|██████████|


,Id_SERVIDOR_PORTAL,NOME,CPF,MATRICULA,COD_TIPO_APOSENTADORIA,TIPO_APOSENTADORIA,DATA_APOSENTADORIA,DESCRICAO_CARGO,COD_UORG_LOTACAO,UORG_LOTACAO,...,DATA_INGRESSO_CARGOFUNCAO,DATA_NOMEACAO_CARGOFUNCAO,DATA_INGRESSO_ORGAO,DOCUMENTO_INGRESSO_SERVICOPUBLICO,DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO,DIPLOMA_INGRESSO_CARGOFUNCAO,DIPLOMA_INGRESSO_ORGAO,DIPLOMA_INGRESSO_SERVICOPUBLICO,ANO,MES
0,3021564,SIDNEY GOMES PEREIRA,***.033.690-**,000****,3,REFORMA POR INVALIDEZ,07/10/1952,Segundo-Sargento,-1,Sem informação,...,01/01/1952,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
1,667339,JOSE REYNALDO SOUZA GOES,***.675.285-**,000****,3,REFORMA POR INVALIDEZ,26/02/1993,Segundo-Tenente,-1,Sem informação,...,01/01/1957,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
2,3612242,RENE GREBOGE DE ALMEIDA,***.895.709-**,000****,2,REFORMA,19/05/1975,Cabo (engajado),-1,Sem informação,...,01/01/1966,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
3,2980660,DOURIVAL TEIXEIRA DOS SANTOS,***.222.985-**,000****,3,REFORMA POR INVALIDEZ,22/11/1972,Terceiro-Sargento,-1,Sem informação,...,01/01/1968,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
4,3127285,LUIZ CARLOS DO CARMO,***.328.899-**,000****,2,REFORMA,21/02/2006,Segundo-Tenente,-1,Sem informação,...,01/01/1968,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169788,734617,PAULO ROBERTO DOS SANTOS,***.037.546-**,099****,2,REFORMA,27/02/2019,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
169789,3454011,RICARDO LUTERMAN,***.049.988-**,099****,2,REFORMA,24/02/2017,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
169790,2439438,ROBERTO SIMAS CORDEIRO,***.705.257-**,099****,5,REFORMA POR DOENÇA,12/11/2004,Capitão-de-Mar-e-Guerra,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4
169791,3373745,RONALDO NETTO PIRES,***.833.237-**,099****,2,REFORMA,19/02/2016,Capitão-de-Fragata,-1,Sem informação,...,07/03/1979,NaN,NaN,0,NaN,NaN,NaN,PORTARIA,2024,4


In [5]:
query = """
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, ORG_LOTACAO as org_lotacao,  
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_abr_militares_reserva_reforma_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, ORG_LOTACAO as org_lotacao, 
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_abr_militares_reserva_reforma_cadastro` 
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, ORG_LOTACAO as org_lotacao,  
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_abr_militares_reserva_reforma_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, ORG_LOTACAO as org_lotacao,  
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_abr_militares_reserva_reforma_cadastro`
               UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria, ORG_LOTACAO as org_lotacao,  
           1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_abr_militares_reserva_reforma_cadastro`
        """  

In [6]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df

Downloading: 100%|██████████|


,ano,mes,tipo_aposentadoria,org_lotacao,total
0,2024,4,REFORMA POR INVALIDEZ,Comando do Exército,1
1,2024,4,REFORMA POR INVALIDEZ,Comando da Aeronáutica,1
2,2024,4,REFORMA,Comando do Exército,1
3,2024,4,REFORMA POR INVALIDEZ,Comando da Aeronáutica,1
4,2024,4,REFORMA,Comando da Aeronáutica,1
...,...,...,...,...,...
831764,2020,4,REFORMA POR DOENÇA,Comando da Marinha,1
831765,2020,4,REFORMA POR DOENÇA,Comando da Marinha,1
831766,2020,4,REFORMA POR DOENÇA,Comando da Marinha,1
831767,2020,4,REFORMA POR DOENÇA,Comando da Marinha,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831769 entries, 0 to 831768
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ano                 831769 non-null  Int64 
 1   mes                 831769 non-null  Int64 
 2   tipo_aposentadoria  831769 non-null  object
 3   org_lotacao         831769 non-null  object
 4   total               831769 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 34.1+ MB


In [8]:
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].str.title()

In [9]:
df['tipo_aposentadoria'].unique()

array(['Reforma Por Invalidez', 'Reforma', 'Reforma Por Doença',
       'Reserva', 'Reserva Ou Reforma Por Decisão Judicial',
       'Reserva Por Doença'], dtype=object)

In [10]:
df['org_lotacao'].unique()

array(['Comando do Exército', 'Comando da Aeronáutica',
       'Comando da Marinha'], dtype=object)

In [14]:
df

,ano,mes,tipo_aposentadoria,org_lotacao,total
0,2024,4,Reforma Por Invalidez,Comando do Exército,1
1,2024,4,Reforma Por Invalidez,Comando da Aeronáutica,1
2,2024,4,Reforma,Comando do Exército,1
3,2024,4,Reforma Por Invalidez,Comando da Aeronáutica,1
4,2024,4,Reforma,Comando da Aeronáutica,1
...,...,...,...,...,...
831764,2020,4,Reforma Por Doença,Comando da Marinha,1
831765,2020,4,Reforma Por Doença,Comando da Marinha,1
831766,2020,4,Reforma Por Doença,Comando da Marinha,1
831767,2020,4,Reforma Por Doença,Comando da Marinha,1


In [15]:
df['org_lotacao'].unique()

array(['Comando do Exército', 'Comando da Aeronáutica',
       'Comando da Marinha'], dtype=object)

In [16]:
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')

In [17]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),        
        bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('org_lotacao','STRING',description='Órgão de lotação'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação')
        ]

In [18]:
table_ref = dataset_ref.table('MILITARES_reserva_reforma_total') 
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result() 


LoadJob<project=repositoriodedadosgpsp, location=US, id=f05e146d-01db-41f8-a182-5eb5b66a7eec>